# Data Mining Course Spark Exercise
## Sharif University of Technology

# Please enter your name below:
# Name: Ilia Hashemi Rad
# Student Number: 99102456

# Section 1: Dataset prepartition
This section you need to download [dataset](https://drive.google.com/file/d/1bRxHQDzPr6wDimbM7b89H47kH8O3YV8Y/view?usp=sharing) in a directory you work. After that run the below cell to untar the datase.

**Note 1: Don't change the below command.**

**Note 2: If you use Windows OS, unzip the dataset manually.**

## Install Pypark & Initialization
Uncomment this section if you use google colab or local pc

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW3") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/31 13:00:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reading the data

In [21]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")

# Showing Data Hierarchical

### Tweet type : Retweeted

In [19]:
import json

def create_tree(tweet_data):
    tree = {}
    
    # Extracting top-level fields
    tree["id"] = tweet_data["id"]
    tree["user"] = {
        "id": tweet_data["user"]["id"],
        "screen_name": tweet_data["user"]["screen_name"],
        "name": tweet_data["user"]["name"],
        "description": tweet_data["user"]["description"]
    }
    tree["tweet_type"] = tweet_data["tweet_type"]
    tree["in_reply_to_status_id_str"] = tweet_data["in_reply_to_status_id_str"]
    tree["in_reply_to_user_id_str"] = tweet_data["in_reply_to_user_id_str"]
    
    # Quoted status
    quoted_status = tweet_data.get("quoted_status")
    if quoted_status:
        tree["quoted_status"] = {
            "id": quoted_status["id"],
        }
    
    # Retweeted status
    retweeted_status = tweet_data.get("retweeted_status")
    if retweeted_status:
        tree["retweeted_status"] = {
            "id": retweeted_status["id"],
            "created_at": retweeted_status["created_at"],
            "user": {
                "id": retweeted_status["user"]["id"],
                "screen_name": retweeted_status["user"]["screen_name"],
                "name": retweeted_status["user"]["name"],
                "description": retweeted_status["user"]["description"]
            }
        }
    
    return tree

# Example for retweeted data
sample_data = json.loads(tweets_rdd.take(10)[0])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))


{
  "id": "1736536753815834746",
  "user": {
    "id": "2808957837",
    "screen_name": "maryaa_maryam",
    "name": "maryam jafari",
    "description": ""
  },
  "tweet_type": "retweeted",
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id_str": null,
  "retweeted_status": {
    "id": "1736379038376120479",
    "created_at": 1702819997,
    "user": {
      "id": "2960042892",
      "screen_name": "Ivar_lathbrug2",
      "name": "آیوار ۲",
      "description": "باورمند به نظام پادشاهی مشروطه با @PahlaviReza #جاویدشاه @PhoenixprjIran     https://t.co/kjMxf53cIQ"
    }
  }
}


### Tweet type : Replied

In [11]:
# Example for replied data
sample_data = json.loads(tweets_rdd.take(10)[3])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))

{
  "id": "1736536759964512322",
  "user": {
    "id": "1406742149048287242",
    "screen_name": "sun_saeid",
    "name": "حق نامه (تلخ)",
    "description": "از زندگی بگیم ولی واقعیتشو"
  },
  "tweet_type": "replied",
  "in_reply_to_status_id_str": "1736356968162378084",
  "in_reply_to_user_id_str": "1643210821068046338"
}


### Tweet type : Generated

In [13]:
# Example for generated data
sample_data = json.loads(tweets_rdd.take(10)[1])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))

{
  "id": "1736536756835512564",
  "user": {
    "id": "22495731",
    "screen_name": "AlArabiya_Fa",
    "name": "العربیه فارسی",
    "description": "«العربيه فارسى» به عنوان بخشی از شبكه «العربيه» در سال 2008 راه‌اندازی شد. \nایمیل تحریریه العربیه فارسی:\neditor.farsi@alarabiya.net\nhttps://t.co/kAD6bIfOgv"
  },
  "tweet_type": "generated",
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id_str": null
}


### Tweet type : Quoted

In [20]:
# Example for quoted data
sample_data = json.loads(tweets_rdd.take(10)[7])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))

{
  "id": "1736536759943450888",
  "user": {
    "id": "1655373597001523200",
    "screen_name": "ScarFace2582",
    "name": "𝐒𝐂𝐀𝐑𝐅𝐀𝐂𝐄 ♔",
    "description": "♧\n#ایران_معبد_ماست #پاینده_ایران #جاویدشاه\n♧"
  },
  "tweet_type": "quoted",
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id_str": null,
  "quoted_status": {
    "id": "1736113075911077948"
  },
  "retweeted_status": {
    "id": "1736536140063109215",
    "created_at": 1702857453,
    "user": {
      "id": "893757772025798656",
      "screen_name": "YaarDabestaani",
      "name": "𓄂✺Yaar - يار ‌‌دبستانی 🏳️‍🌈",
      "description": "Human rights activist, 🌈LGBTQ🏳️‍🌈, animal lover🐕‍🦺🐈‍⬛🦥, Atheist, 👩🏿‍💼=🧑🏻‍💼=👩🏼‍💼=🧑🏿‍💼 #MahsaAmini,  My inner child: @bri30s"
    }
  }
}


# Determining Users' Rating for Tweets

In [28]:
import json

tweets_rdd = sc.textFile("twitter_data_v2.jsonl")

# Parse JSON strings to Python dictionaries
tweets_json = tweets_rdd.map(json.loads)
del tweets_rdd

# Filter out generated tweets
relevant_tweets = tweets_json.filter(lambda x: x['tweet_type'] in ["retweeted", "replied", "quoted"])


# Extract tweet rates for each user
def extract_tweet_rates(tweet):
    user_id = tweet.get("user", {}).get("id", None)
    tweet_type = tweet.get("tweet_type", None) 
    if tweet_type == 'replied':
        in_reply_to_status_id_str = tweet.get("in_reply_to_status_id_str", None)
        return (user_id, in_reply_to_status_id_str, 1)
    elif tweet_type == 'retweeted':
        retweeted_tweet_id = tweet.get("retweeted_status", {}).get("id", None)
        return (user_id, retweeted_tweet_id, 2)
    else:
        quoted_tweet_id = tweet.get("quoted_status", {}).get("id", None)
        return (user_id, quoted_tweet_id, 3)

user_rating = relevant_tweets.map(extract_tweet_rates)

user_rating.take(8)

[('2808957837', '1736379038376120479', 2),
 ('165605880', '1736495024190136471', 2),
 ('1406742149048287242', '1736356968162378084', 1),
 ('1726367085377544192', '1736236576517054856', 2),
 ('1254572621208920064', '1736509619457974401', 2),
 ('1655373597001523200', '1736113075911077948', 3),
 ('1537114095090016257', '1736471343267234237', 2),
 ('716909714643283968', '1736444294041620617', 3)]

# Creating User Tweet Matrix

In [30]:
# Function to create a matrix of users-tweets and their respective ratings
def create_matrix(rdd):
    user_tweet_matrix = rdd.map(lambda x: (x[0], [(x[1], x[2])])) \
        .reduceByKey(lambda x, y: x + y) \
        .map(lambda x: (x[0], dict(x[1])))
    return user_tweet_matrix

user_tweet_matrix = create_matrix(user_rating)

user_tweet_matrix.take(5)[2:4]

[('1573728368851296261',
  {'1736507311852519557': 1,
   '1736113075911077948': 3,
   '1736370199815680482': 2,
   '1736382032760963570': 2,
   '1736459949956362310': 1,
   '1736515482469966144': 1,
   '1736178656278389027': 1,
   '1734522655452041420': 2,
   '1736096711972475174': 1,
   '1736015535807050185': 2,
   '1734420560644616543': 2,
   '1736329393696284675': 1,
   '1734508776776949774': 2,
   '1736635791168991344': 2,
   '1734646567855063280': 2,
   '1735866847873106173': 1,
   '1734098972934021140': 1,
   '1734181647522603254': 2,
   '1734967652248895738': 2,
   '1734109226199593049': 1,
   '1736510537033277547': 2,
   '1734673975391719496': 1,
   '1734665832343425153': 1,
   '1734620386929647834': 1,
   '1734577106175091029': 2,
   '1734278587036115269': 1,
   '1736612534973600118': 1,
   '1736722921052135478': 1,
   '1736654857388310812': 1,
   '1736359116073775210': 1,
   '1733974468639301807': 2,
   '1736863861763195060': 1,
   '1736862227389403587': 2,
   '17367586891245

# Choosing One target User And Showing his Rated Tweets

In [69]:
# Choose a target user to suggest tweets
target_user_id = '165605880'

# Determine ratings of the target user (corresponding column in the matrix for user)
target_user_ratings = user_tweet_matrix.lookup(target_user_id)[0]

# Create the list of tweets which the user rated as (Tweet ID, Tweet text)
user_rated_tweets = tweets_json.filter(lambda x: x['id'] in list(target_user_ratings.keys())).map(lambda x: (x['id'], x['text'])).collect()

# Show some samples of user's favorite tweets
for tweet in user_rated_tweets[:5] + user_rated_tweets[6:8]:
    print(f'Rate of user to the tweet: {target_user_ratings[tweet[0]]}\n')
    print('Tweet Text:')
    print(tweet[1])
    print('\n\n')
    

Rate of user to the tweet: 2

Tweet Text:
⚡️#فوری⚡️ 
 سفر #رئیسی_جلاد۶۷  به #ژنو با تلاشهای #مجاهدین_خلق_ایران و شورشیاران و حامیان مقاومت و شکایت رسمی لغو شد   #مبارک #مبارک 👏👏 #No2Raisi اینهم شادی همه کوشندگان برای آزادی که «گل عظما» رو اذیت کردن🤣🤣🤣 
#كانونهای_شورشی 🔥🔥🔥🔥🔥
https://t.co/mIeqYL2bTB



Rate of user to the tweet: 3

Tweet Text:
👏👏👏👏🙏💐
درود براو بخاطرپاکبازی وهمیاری صادقانهٔ اوبرای هدف مشترک مبارزه علیه دشمنان وخائنین دروغگوبه مردم وبرای دفاع ازحق مردم شریف وشجاع ایران، درود👏👏👏🔥🔥🔥
تا به ابد نامش بر تارک مبارزات حق جویانهٔ #مردم_ایران پاینده باد
#مرگ_بر_ستمگر_چه_شاه_باشه_چه_رهبر 
#مهسا_امينی‌



Rate of user to the tweet: 1

Tweet Text:
🔻فوری؛‌ فوری؛‌ فوری؛‌
🔸خبرگزاری فرانسه: رئیس جمهور ایران در آستانه سفر به سوئیس هدف شکایت «جنایت علیه بشریت» قرار گرفت

🔸طبق شکایت حقوقی ثبت شده در روز دوشنبه رئیس جمهور رژیم ایران باید هنگام ورودش به سوئیس این هفته به اتهام جنایت علیه بشریت مرتبط با قتل‌عام مخالفان در سال ۱۹۸۸ دستگیر شود.

🔸در شکایت از دادستان عمومی فدرال سوئیس خواسته شده ا

# Finding Similar Users

In [70]:
from prettytable import PrettyTable

# Function to calculate Pearson correlation coefficient
def pearson_corr(user1_ratings, user2_ratings):
    common_tweets = set(user1_ratings.keys()) & set(user2_ratings.keys())
    if not common_tweets:
        return 0.0

    sum_xy = sum(user1_ratings[tweet] * user2_ratings[tweet] for tweet in common_tweets)
    sum_x = sum(user1_ratings[tweet] for tweet in common_tweets)
    sum_y = sum(user2_ratings[tweet] for tweet in common_tweets)

    sum_x_squared = sum(user1_ratings[tweet] ** 2 for tweet in common_tweets)
    sum_y_squared = sum(user2_ratings[tweet] ** 2 for tweet in common_tweets)

    n = len(common_tweets)

    numerator = sum_xy - (sum_x * sum_y / n)
    denominator = ((sum_x_squared - (sum_x ** 2 / n)) * (sum_y_squared - (sum_y ** 2 / n))) ** 0.5

    if denominator == 0:
        return 0.0

    return numerator / denominator

# Function to find similar users based on Pearson correlation
def find_similar_users(user_id, user_tweet_matrix, NumSim):
    target_user_ratings = user_tweet_matrix.lookup(user_id)[0]
    similarities = user_tweet_matrix.filter(lambda x: x[0] != user_id) \
        .map(lambda x: (x[0], pearson_corr(target_user_ratings, x[1])))
    
    # Get top k similar users
    similar_users = similarities.takeOrdered(NumSim, key=lambda x: -x[1])

    return similar_users

# The target user
target_user_id = '165605880'

# Number of top similar users which we're gonna find
NumberOfSimilarUsrs = 50

# Find similar users
similar_users = find_similar_users(target_user_id, user_tweet_matrix, NumberOfSimilarUsrs)

table = PrettyTable()
table.field_names = ["User ID", "Similarity"]

for user, similarity in similar_users:
    table.add_row([user, similarity])

print(table)


+---------------------+--------------------+
|       User ID       |     Similarity     |
+---------------------+--------------------+
|  853963316393365505 |        1.0         |
| 1286033063173070851 |        1.0         |
|      2682190189     |        1.0         |
| 1412491972934356996 |        1.0         |
| 1679386714480029697 |        1.0         |
|       75820019      |        1.0         |
|  858296771390341120 |        1.0         |
|  963777541910577152 |        1.0         |
|      4447515796     |        1.0         |
|  947162985549778945 |        1.0         |
| 1473130308992180227 |        1.0         |
| 1512665397262172167 |        1.0         |
| 1311708366805389315 |        1.0         |
| 1458078338954715139 |        1.0         |
| 1602640571708973057 |        1.0         |
|      1641243072     |        1.0         |
|      2250579744     |        1.0         |
|      294130276      |        1.0         |
| 1339649759490039809 |        1.0         |
| 13001317

# Recommend Tweets to target user based on his Similar User Preferences

In [81]:
# Function to predict missing ratings for a given user
def predict_ratings(user_id, user_tweet_matrix, similar_users):
    target_user_ratings = user_tweet_matrix.lookup(user_id)

    if not target_user_ratings:
        return {}

    target_user_ratings = target_user_ratings[0]

    # Extract tweet IDs rated by similar users
    similar_users_ids = [similar_user for similar_user, _ in similar_users]
    similar_users_ids.append(user_id)

    # Filter the original RDD to get only the rows corresponding to similar users
    similar_tweet_matrix = user_tweet_matrix.filter(lambda x: x[0] in similar_users_ids).collect()

    # Create a dictionary for quick access to ratings of similar users
    similar_users_ratings_dict = dict(similar_tweet_matrix)

    # Find the set of all tweet IDs rated by similar users
    similar_users_tweets = set()
    for similar_user_id in similar_users_ids:
        if similar_user_id in similar_users_ratings_dict:
            similar_users_tweets.update(similar_users_ratings_dict[similar_user_id].keys())

    # Remove the tweets that the target user has already rated
    similar_users_tweets -= set(target_user_ratings.keys())

    predicted_ratings = {}

    for tweet_id in similar_users_tweets:
        numerator = 0.0
        denominator = 0.0

        for similar_user_id, similarity in similar_users:
            if tweet_id in similar_users_ratings_dict.get(similar_user_id, {}):
                numerator += similarity * similar_users_ratings_dict[similar_user_id][tweet_id]
                denominator += abs(similarity)

        if denominator != 0:
            predicted_ratings[tweet_id] = numerator / denominator
        else:
            predicted_ratings[tweet_id] = None

    return predicted_ratings


predicted_ratings = predict_ratings(target_user_id, user_tweet_matrix, similar_users)

# Print the top recommended tweets based on predicted ratings
top_recommended_tweets = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)

table = PrettyTable()
table.field_names = ["Tweet ID", "Predicted Rating"]

top_recom_ids = []
i = 1
for tweet_id, rating in top_recommended_tweets:
    if i in range(1,15) or i in range(115, 130) or i in range(1822, 1852):
        top_recom_ids.append(tweet_id)
    i = i+1
    table.add_row([tweet_id, rating])

print(table)


+---------------------+--------------------+
|       Tweet ID      |  Predicted Rating  |
+---------------------+--------------------+
| 1734922650844020829 | 3.0000000000000004 |
| 1735317486080315505 | 3.0000000000000004 |
| 1734548053062045939 | 3.0000000000000004 |
| 1733604306761236932 | 3.0000000000000004 |
| 1735582412736721376 | 3.0000000000000004 |
| 1735605516770947141 | 3.0000000000000004 |
| 1734657841896575294 | 3.0000000000000004 |
| 1736363452560445906 | 3.0000000000000004 |
| 1734815968080064929 | 3.0000000000000004 |
| 1733528923902820357 | 3.0000000000000004 |
| 1735999525586612442 |        3.0         |
| 1733953865291055555 |        3.0         |
| 1736679948444680664 |        3.0         |
| 1735959872611463566 |        3.0         |
| 1730311120123265375 |        3.0         |
| 1733018278560256231 |        3.0         |
| 1736088911154127257 |        3.0         |
| 1736282607132832019 |        3.0         |
| 1733956917129625991 |        3.0         |
| 17350760

## Evaluation of Recommended Tweets

In [82]:
# Create the list of tweets which is recommended to user (Tweet ID, Tweet text)
user_recommended_tweets = tweets_json.filter(lambda x: x['id'] in top_recom_ids).map(lambda x: (x['id'], x['text'])).collect()

top_recommended_tweets = dict(top_recommended_tweets)

# Show some recommended tweets to user and their corresponding recommendation score(predication rate)
for tweet in user_recommended_tweets:
    print(f'Rate of user to the tweet: {top_recommended_tweets[tweet[0]]}\n')
    print('Tweet Text:')
    print(tweet[1])
    print('\n\n')

Rate of user to the tweet: 2.9999999999999996

Tweet Text:
ببین یاسمنگولا چقدر سلیطه است که حتی نازنین‌بنیادی هم تیکه سنگین بهش میندازه 😂 https://t.co/Y8h9um0LdR



Rate of user to the tweet: 1.5691904145692734

Tweet Text:
سرب گلوله‌ای که دهه ۶۰ منافقین به پدرم شلیک کردند هنوز در گونه‌اش موجود است.
امروز در دادگاه رسیدگی به جنایات گروهک رجوی حضور داشتم.‌ ملت ایران با منافقین پدرکشتگی دارد... https://t.co/09WNDF6vYh



Rate of user to the tweet: 3.0000000000000004

Tweet Text:
🔴 بیانیه‌ی انجمن سبز چیا در واکنش به انفجار در جنگل‌های #زاگرس: «کوه خواری با انفجار مهیب و ترسناک / طبیعت و جنگل کیلویی چند؟!»

#زن_زندگى_آزادى #محیط_زیست 
«انفجاری مهیب با این وسعت در میان جنگل‌های زاگروس بدون شک اعلان جنگ است، جنگی بە تمام معنا علیه جنگل‌ها و منابع طبیعی این سرزمین که جزو اندک سرمایه ی عمومی مردمان این دیار است.
سالیان درازیست که کارخانه شن و ماسه اطراف روستای " ساوا " از توابع بخش خاو میرآباد شهرستان مریوان بدون در نظر گرفتن اصول زیست محیطی و به دلخواه خود اقدام به کوه خواری نموده و اینگونه ک